# Alinhamento de sequências

## Como determinar quão similares duas sequências são?

Vimos na apresentação que em 1970 foi criado o algoritmo de Needleman-Wunsch, capaz de realizar um alinhamento global, isto é, de sequência inteira, entre um par de proteínas. Mas como ele faz isso?

### Distância de Hamming

Para começar, vamos olhar para a distância de hamming entre duas sequências de proteína. Em bioinformática, tais sequências podem ser representadas por uma *string* contendo as letras que representam cada aminoácido. Sendo assim, tomemos as sequências `seqA`, `seqB` e `seqC`.

In [1]:
from scipy.spatial.distance import hamming
import skbio

# NCBI Reference Sequence: NP_000508.1 (human hemoglobin subunit A)
seqA = skbio.Protein("MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR")

# NCBI Reference Sequence: NP_001004376.1 (chicken hemoglobin subunit A)
seqB = skbio.Protein("MVLSAADKNNVKGIFTKIAGHAEEYGAETLERMFTTYPPTKTYFPHFDLSHGSAQIKGHGKKVVAALIEAANHIDDIAGTLSKLSDLHAHKLRVDPVNFKLLGQCFLVVVAIHHPAALTPEVHASLDKFLCAVGTVLTAKYR")

# GenBank: QFF91579.1 (sei whale hemoglobin subunit A)
seqC = skbio.Protein("MVLFPADKSNVKATWAKIGNHGAEYGAEALERMFMNFPSTKTYFPHFDLGHDSAQVKGHGKKVADALTKAAGHMDNLLDALSDLSDLHAHKLRVDPVNFKLLSHCLLVTLALHLPAEFTPSVHASLDKFLASVSTVLTSKYR")

print(seqA)
print(seqB)
print(seqC)

MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR
MVLSAADKNNVKGIFTKIAGHAEEYGAETLERMFTTYPPTKTYFPHFDLSHGSAQIKGHGKKVVAALIEAANHIDDIAGTLSKLSDLHAHKLRVDPVNFKLLGQCFLVVVAIHHPAALTPEVHASLDKFLCAVGTVLTAKYR
MVLFPADKSNVKATWAKIGNHGAEYGAEALERMFMNFPSTKTYFPHFDLGHDSAQVKGHGKKVADALTKAAGHMDNLLDALSDLSDLHAHKLRVDPVNFKLLSHCLLVTLALHLPAEFTPSVHASLDKFLASVSTVLTSKYR


In [2]:
print(hamming(seqA, seqC))
print(hamming(seqB, seqC))

0.16901408450704225
0.31690140845070425


Aqui vemos que, para a cadeia A da hemoglobina, a distância de hamming entre a sequência humana (`seqA`) e a sequência de baleia (`seqC`) é menor do que aquela entre a sequência de galinha (`seqB`) e a sequência de baleia (`seqC`). 

- O que isso pode significar, em termos evolutivos?

![galinha ciscando](https://media.giphy.com/media/l3vR9IEU6nYAmZyoM/giphy.gif)

Essa comparação utilizando a distância de Hamming pode ser interessante quando falamos de mudanças causadas apenas por substituições de aminoácidos. Porém, no DNA podem acontecer:
- substituições
- inserções
- deleções

E há ainda outras formas de categorizar as mutações possíveis e existentes. O que aconteceria com essa métrica caso ocorresse uma deleção no último aminoácido da sequência?

In [3]:
# NCBI Reference Sequence: XP_028905054.1 (platypus hemoglobin subunit A); 
# Cadeia A da Hemoglobina de Ornitorrinco
# Possui um aminoácido a menos, então vamos adicionar um hífen '-'
# assim poderemos calculara distância de Hamming

seqD = skbio.Protein("MLTDAEKKEVTALWGKAAGHGEEYGAEALERLFQAFPTTKTYFSHFDLSHGSAQIKAHGKKVADALSTAAGHFDDMDSALSALSDLHAHKLRVDPVNFKLLAHCILVVLARHCPGEFTPSAHAAMDKFLSKVATVLTSKYR-")

print(hamming(seqA, seqD))
print(hamming(seqB, seqD))

0.9084507042253521
0.9225352112676056


Aqui, apesar de uma relação de **homologia** conhecida entre esses genes, a distância de Hamming saltou em relação os valores anteriores. Aqui os valores sugerem que a proteína em questão é mais similar entre humanos e ornitorrincos, ambos mamíferos, o que faz sentido. Mas os valores são muito mais altos. 

Vamos olhar um pouco para as sequências...

In [4]:
print(seqA[0:10],'...',seqA[-10:]) # humano
print(seqB[0:10],'...',seqB[-10:]) # galinha
print(seqC[0:10],'...',seqC[-10:]) # baleia
print(seqD[0:10],'...',seqD[-10:]) # ornitorrinco

MVLSPADKTN ... VSTVLTSKYR
MVLSAADKNN ... VGTVLTAKYR
MVLFPADKSN ... VSTVLTSKYR
MLTDAEKKEV ... ATVLTSKYR-


O que se pode notar é que o fim das sequências se parece, todas terminando em `TVLTAKYR`, enquanto que no início das sequências, uma valina (`V`) parece ter *deletada* da sequência de ornitorrinco. O que acontece com a distância de hamming se mudarmos o hífen de lugar?

In [5]:
# NCBI Reference Sequence: XP_028905054.1 (platypus hemoglobin subunit A); 
# Cadeia A da Hemoglobina de Ornitorrinco
# Possui um aminoácido a menos, então vamos adicionar um hífen '-'
# Esse hífen será adicionado na segunda posição para alinhar manualmente as seq

seqD_alinhada = skbio.Protein("M-LTDAEKKEVTALWGKAAGHGEEYGAEALERLFQAFPTTKTYFSHFDLSHGSAQIKAHGKKVADALSTAAGHFDDMDSALSALSDLHAHKLRVDPVNFKLLAHCILVVLARHCPGEFTPSAHAAMDKFLSKVATVLTSKYR")

print(hamming(seqA, seqD_alinhada)) # humano x ornitorrinco
print(hamming(seqB, seqD_alinhada)) # galinha x ornitorrinco

0.2746478873239437
0.34507042253521125


In [6]:
print(hamming(seqA, seqD_alinhada)) # humano x ornitorrinco
print(hamming(seqA, seqC)) # humano x baleia

0.2746478873239437
0.16901408450704225


Esses valores agora são muito mais condizentes com os valores esperados para sequências próximas. Além disso, vemos que a distância entre as sequências de humanos e ornitorrincos é maior que aquela entre as humanos e baleia. Esse resultado se apoia ao que conhecemos sobre a história evolutiva dos mamíferos: ornitorrincos surgiram como espécie antes de baleias, e estas antes de humanos.

> Comparar sequências do DNA, do RNA e das proteínas de várias espécies pode ser usado para construir um *relógio molecular* que aponta as distâncias relativas entre essas espécies.

Para fins de visualização, observe agora as sequências humana e de ornitorrinco, com o alinhamento manual.

## Um algoritmo para o alinhamento global de duas sequências

Agora que entendemos a necessidade de um alinhamento de sequências que leve em conta os possíveis eventos de mutação das sequências ao longo da evolução das espécies, vamos entender como funciona um algorimo de alinhamento de pares de sequências.


In [7]:
print(seqA)
print(seqD_alinhada)

MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR
M-LTDAEKKEVTALWGKAAGHGEEYGAEALERLFQAFPTTKTYFSHFDLSHGSAQIKAHGKKVADALSTAAGHFDDMDSALSALSDLHAHKLRVDPVNFKLLAHCILVVLARHCPGEFTPSAHAAMDKFLSKVATVLTSKYR


In [8]:
from skbio import DNA

seqA = DNA("ACCGGTGGAACCGGTAACACCCAC")
seqB = DNA("ACCGGTAACCGGTTAACACCCAC")

Com nossas sequências fictícias podemos criar então uma matriz para os alinhamentos. O número de colunas e linhas depende do comprimento das sequências. Esta matriz será inicializada com zeros.

In [9]:
import numpy as np

num_linhas = len(seqB)
num_colunas = len(seqA)
matriz = np.zeros(shape=(num_linhas, num_colunas), dtype=int)

In [10]:
from intro_bioinfo_unir.iab import show_F

show_F(seqA, seqB, matriz) #exibe a matriz 

,A,C,C,G,G,T,G,G,A,A,C,C,G,G,T,A,A,C,A,C,C,C,A,C
A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
G,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
T,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
C,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


One há correspondências idênticas entre as duas sequências (linhas e colunas), substituir zero por 1.

In [11]:
for row_number, row_character in enumerate(seqB):
    for col_number, col_character in enumerate(seqA):
        if row_character == col_character:
            matriz[row_number, col_number] = 1

show_F(seqA, seqB, matriz, hide_zeros=True)

,A,C,C,G,G,T,G,G,A,A,C,C,G,G,T,A,A,C,A,C,C,C,A,C
A,1,,,,,,,,1,1,,,,,,1,1,,1,,,,1,
C,,1,1,,,,,,,,1,1,,,,,,1,,1,1,1,,1
C,,1,1,,,,,,,,1,1,,,,,,1,,1,1,1,,1
G,,,,1,1,,1,1,,,,,1,1,,,,,,,,,,
G,,,,1,1,,1,1,,,,,1,1,,,,,,,,,,
T,,,,,,1,,,,,,,,,1,,,,,,,,,
A,1,,,,,,,,1,1,,,,,,1,1,,1,,,,1,
A,1,,,,,,,,1,1,,,,,,1,1,,1,,,,1,
C,,1,1,,,,,,,,1,1,,,,,,1,,1,1,1,,1
C,,1,1,,,,,,,,1,1,,,,,,1,,1,1,1,,1


Agora vamos buscar onde esta a maior sequência de células com valores diferentes de zero, aqui chamadas diagonais. Biologicamente, essas são as porções em que as duas sequências seguem idênticas, sem substituições ou deleções (ou inserções).

In [12]:
# copia-se a matriz original para ter um backup
matriz_somada = matriz.copy()
# iteração por todas as células a partir da
# segunda linha e segunda coluna
for i in range(1, matriz_somada.shape[0]):
    for j in range(1, matriz_somada.shape[1]):
        # se o valor na célula for maior que zero
        # adicionar o valor da célula que estiver acima
        # e o valor da célula que estiver à esquerda
        if matriz_somada[i, j] > 0:
            matriz_somada[i, j] += matriz_somada[i-1][j-1]

show_F(seqA, seqB, matriz_somada, hide_zeros=True)

,A,C,C,G,G,T,G,G,A,A,C,C,G,G,T,A,A,C,A,C,C,C,A,C
A,1,,,,,,,,1,1,,,,,,1,1,,1,,,,1,
C,,2,1,,,,,,,,2,1,,,,,,2,,2,1,1,,2
C,,1,3,,,,,,,,1,3,,,,,,1,,1,3,2,,1
G,,,,4,1,,1,1,,,,,4,1,,,,,,,,,,
G,,,,1,5,,1,2,,,,,1,5,,,,,,,,,,
T,,,,,,6,,,,,,,,,6,,,,,,,,,
A,1,,,,,,,,1,1,,,,,,7,1,,1,,,,1,
A,1,,,,,,,,1,2,,,,,,1,8,,1,,,,1,
C,,2,1,,,,,,,,3,1,,,,,,9,,2,1,1,,2
C,,1,3,,,,,,,,1,4,,,,,,1,,1,3,2,,1


Onde está a maior diagonal somada?

![patolino rocurando](https://c.tenor.com/z8r_8_sMbOMAAAAC/searching-investigation.gif)

In [13]:
maior_diagonal = matriz_somada.max()
print("A maior diagonal tem %d nucleotídeos de comprimento." % maior_diagonal)

A maior diagonal tem 10 nucleotídeos de comprimento.


O que isso significa em termos de alinhamento? Fizemos um alinhamento manual anteriormente, mas e agora, com essa tabela, onde adicionar os hífens?

De maneira resumida, o algoritmo vai 
> 1. buscar a maior diagonal 
> 2. escrever a sequência a partir do último nucleotídeo correspondente para as duas sequências;
> 3. seguir voltando na sequência (subindo a diagonal) até o fim da maior diagonal

Quando a diagonal quebrar, as ações seguintes incluem...

> 4. buscar a segunda maior diagonal acima ou à esquerda
> 5. adicionar hífens/gaps/intervalos para cada célula que se teve que pular até essa diagonal: se foi para cima, o gap vai para a sequência correspondente às colunas da matriz, se foi para a esquerda, o gap vai para a sequência correspondente às linhas da matriz.
> 6. calcula-se um escore para o alinhamento

Para essa visualização, vamos simplesmente adicionar 1 para cada correspondência e -1 para cada substituição, sem valor específico para os gaps.

Esses são dois alinhamentos possíveis:

```bash
Alinhamento 1 (score: 19)

ACCGGTGGAACCGG-TAACACCCAC
ACCGGT--AACCGGTTAACACCCAC

Alinhamento 2 (score: 8)

ACCGGTGGAACCGGTAACACCCAC
ACCGGT--------TAACACCCAC
```

> Discutam 
- Como os dois alinhamentos apresentados podem ser relevantes biologicamente?
- Quais são as limitações do escore calculado?

![Conversem](https://media0.giphy.com/media/26FPJGjhefSJuaRhu/giphy.gif?cid=ecf05e47rf63k20r0c6lu1y1uwwcsk2vx123og5vqqu1u19n&rid=giphy.gif&ct=g)

### *"Todos os modelos estão errados, mas alguns são úteis."* **George Box**

Existe toda uma maquinaria celular para a manutenção da fidelidade da replicação (síntese de DNA), existem ainda mecanismos para garantir essa fidelidade na síntese de RNA e proteína. Muitas mutações prejudicam a regulação de um gene, dificultando sua expressão ou ainda impedem ou reduzem a eficiência de uma proteína que possa ser traduzida. Por isso, mutações não ocorrem em todas as posições e a todo momento.

![Atividade de Correção da DNA Polimerase](https://cdn.kastatic.org/ka-perseus-images/fd025fc7b765b7df394990fa014b93b70c94eba6.png)

O pequeno número de mutações que ocorre, é passado adiante e que confere vantagem (ou que não prejudica) os indivíduos vai se espalhando ou simplesmente permanece na população ao longo de gerações.

De modo geral não se atribui a mesma pontuação, apenas com sinais invertidos, para correspondências (matches) e substituições (mismatches). Além disso, a presença de intervalos nas sequências, indicando inserções ou deleções em dois ou mais nucleotídeos (ou aminoácidos) também pode ter um impacto severo nos seres vivos, devendo possuir uma "punição" (valor negativo) próprio.

> Assim, ao fazer alinhamentos você deve lembrar que cada resultado é gerado por um algoritmo (ou vários) que tem parâmetros específicos (e modificáveis) e que cada análise parte de pressupostos que devem ser conhecidos por você. Além disso, alguns algoritmos são mais adequados quando se tem muitos alinhamentos (e menos tempo para cada um).

## As matrizes de substituição

Para proteínas, utilizar um único valor para correspondências e um outro para substituições é ainda pior. Como vimos, aminoácidos são moléculas com cadeias laterais que lhes conferem características físico-químicas próprias, e podem ser agrupados de acordo com elas.

Você deve se lembrar da matriz de substituição
PAM1 calculada por Margaret Dayhoff e colaboradores. Outras matrizes foram produzidas com o mesmo propósito; há matrizes PAM obtida por operações com a primeira matriz PAM, e matrizes obtidas por outras vias, como a BLOSUM. 

Enquanto a PAM é uma matriz de substituição gerada pelo alinhamento de proteínas homólogas, pertencentes a um número de famílias, a matriz BLOSUM foi gerada olhando para substituições que ocorrem em regiões muito conservadas identificadas para vários alinhamentos. Diferentes matrizes podem ser utilizadas, a depender do objetivo da análise:

![Variações nas divergências das sequências usadas](https://resources.qiagenbioinformatics.com/manuals/clcgenomicsworkbench/650/pam-blosum.png)

Neste tutorial vamos usar uma BLOSUM50.

In [14]:
blosum50 = {'A': {'A': 5, 'C': -1, 'D': -2, 'E': -1, 'F': -3, 'G': 0, 'H': -2, 'I': -1, 'K': -1, 'L': -2, 'M': -1, 'N': -1, 'P': -1, 'Q': -1, 'R': -2, 'S': 1, 'T': 0, 'V': 0, 'W': -3, 'Y': -2},
'C': {'A': -1, 'C': 13, 'D': -4, 'E': -3, 'F': -2, 'G': -3, 'H': -3, 'I': -2, 'K': -3, 'L': -2, 'M': -2, 'N': -2, 'P': -4, 'Q': -3, 'R': -4, 'S': -1, 'T': -1, 'V': -1, 'W': -5, 'Y': -3},
'D': {'A': -2, 'C': -4, 'D': 8, 'E': 2, 'F': -5, 'G': -1, 'H': -1, 'I': -4, 'K': -1, 'L': -4, 'M': -4, 'N': 2, 'P': -1, 'Q': 0, 'R': -2, 'S': 0, 'T': -1, 'V': -4, 'W': -5, 'Y': -3},
'E': {'A': -1, 'C': -3, 'D': 2, 'E': 6, 'F': -3, 'G': -3, 'H': 0, 'I': -4, 'K': 1, 'L': -3, 'M': -2, 'N': 0, 'P': -1, 'Q': 2, 'R': 0, 'S': -1, 'T': -1, 'V': -3, 'W': -3, 'Y': -2},
'F': {'A': -3, 'C': -2, 'D': -5, 'E': -3, 'F': 8, 'G': -4, 'H': -1, 'I': 0, 'K': -4, 'L': 1, 'M': 0, 'N': -4, 'P': -4, 'Q': -4, 'R': -3, 'S': -3, 'T': -2, 'V': -1, 'W': 1, 'Y': 4},
'G': {'A': 0, 'C': -3, 'D': -1, 'E': -3, 'F': -4, 'G': 8, 'H': -2, 'I': -4, 'K': -2, 'L': -4, 'M': -3, 'N': 0, 'P': -2, 'Q': -2, 'R': -3, 'S': 0, 'T': -2, 'V': -4, 'W': -3, 'Y': -3},
'H': {'A': -2, 'C': -3, 'D': -1, 'E': 0, 'F': -1, 'G': -2, 'H': 10, 'I': -4, 'K': 0, 'L': -3, 'M': -1, 'N': 1, 'P': -2, 'Q': 1, 'R': 0, 'S': -1, 'T': -2, 'V': -4, 'W': -3, 'Y': 2},
'I': {'A': -1, 'C': -2, 'D': -4, 'E': -4, 'F': 0, 'G': -4, 'H': -4, 'I': 5, 'K': -3, 'L': 2, 'M': 2, 'N': -3, 'P': -3, 'Q': -3, 'R': -4, 'S': -3, 'T': -1, 'V': 4, 'W': -3, 'Y': -1},
'K': {'A': -1, 'C': -3, 'D': -1, 'E': 1, 'F': -4, 'G': -2, 'H': 0, 'I': -3, 'K': 6, 'L': -3, 'M': -2, 'N': 0, 'P': -1, 'Q': 2, 'R': 3, 'S': 0, 'T': -1, 'V': -3, 'W': -3, 'Y': -2},
'L': {'A': -2, 'C': -2, 'D': -4, 'E': -3, 'F': 1, 'G': -4, 'H': -3, 'I': 2, 'K': -3, 'L': 5, 'M': 3, 'N': -4, 'P': -4, 'Q': -2, 'R': -3, 'S': -3, 'T': -1, 'V': 1, 'W': -2, 'Y': -1},
'M': {'A': -1, 'C': -2, 'D': -4, 'E': -2, 'F': 0, 'G': -3, 'H': -1, 'I': 2, 'K': -2, 'L': 3, 'M': 7, 'N': -2, 'P': -3, 'Q': 0, 'R': -2, 'S': -2, 'T': -1, 'V': 1, 'W': -1, 'Y': 0},
'N': {'A': -1, 'C': -2, 'D': 2, 'E': 0, 'F': -4, 'G': 0, 'H': 1, 'I': -3, 'K': 0, 'L': -4, 'M': -2, 'N': 7, 'P': -2, 'Q': 0, 'R': -1, 'S': 1, 'T': 0, 'V': -3, 'W': -4, 'Y': -2},
'P': {'A': -1, 'C': -4, 'D': -1, 'E': -1, 'F': -4, 'G': -2, 'H': -2, 'I': -3, 'K': -1, 'L': -4, 'M': -3, 'N': -2, 'P': 10, 'Q': -1, 'R': -3, 'S': -1, 'T': -1, 'V': -3, 'W': -4, 'Y': -3},
'Q': {'A': -1, 'C': -3, 'D': 0, 'E': 2, 'F': -4, 'G': -2, 'H': 1, 'I': -3, 'K': 2, 'L': -2, 'M': 0, 'N': 0, 'P': -1, 'Q': 7, 'R': 1, 'S': 0, 'T': -1, 'V': -3, 'W': -1, 'Y': -1},
'R': {'A': -2, 'C': -4, 'D': -2, 'E': 0, 'F': -3, 'G': -3, 'H': 0, 'I': -4, 'K': 3, 'L': -3, 'M': -2, 'N': -1, 'P': -3, 'Q': 1, 'R': 7, 'S': -1, 'T': -1, 'V': -3, 'W': -3, 'Y': -1},
'S': {'A': 1, 'C': -1, 'D': 0, 'E': -1, 'F': -3, 'G': 0, 'H': -1, 'I': -3, 'K': 0, 'L': -3, 'M': -2, 'N': 1, 'P': -1, 'Q': 0, 'R': -1, 'S': 5, 'T': 2, 'V': -2, 'W': -4, 'Y': -2},
'T': {'A': 0, 'C': -1, 'D': -1, 'E': -1, 'F': -2, 'G': -2, 'H': -2, 'I': -1, 'K': -1, 'L': -1, 'M': -1, 'N': 0, 'P': -1, 'Q': -1, 'R': -1, 'S': 2, 'T': 5, 'V': 0, 'W': -3, 'Y': -2},
'V': {'A': 0, 'C': -1, 'D': -4, 'E': -3, 'F': -1, 'G': -4, 'H': -4, 'I': 4, 'K': -3, 'L': 1, 'M': 1, 'N': -3, 'P': -3, 'Q': -3, 'R': -3, 'S': -2, 'T': 0, 'V': 5, 'W': -3, 'Y': -1},
'W': {'A': -3, 'C': -5, 'D': -5, 'E': -3, 'F': 1, 'G': -3, 'H': -3, 'I': -3, 'K': -3, 'L': -2, 'M': -1, 'N': -4, 'P': -4, 'Q': -1, 'R': -3, 'S': -4, 'T': -3, 'V': -3, 'W': 15, 'Y': 2},
'Y': {'A': -2, 'C': -3, 'D': -3, 'E': -2, 'F': 4, 'G': -3, 'H': 2, 'I': -1, 'K': -2, 'L': -1, 'M': 0, 'N': -2, 'P': -3, 'Q': -1, 'R': -1, 'S': -2, 'T': -2, 'V': -1, 'W': 2, 'Y': 8}}

aas = list(blosum50.keys())
aas.sort()
data = []
for aa1 in aas:
    row = []
    for aa2 in aas:
        row.append(blosum50[aa1][aa2])
    data.append(row)

aa_labels = ''.join(aas)

In [15]:
from intro_bioinfo_unir.iab import show_substitution_matrix

show_substitution_matrix(aa_labels, data)

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
A,5,-1,-2,-1,-3,0,-2,-1,-1,-2,-1,-1,-1,-1,-2,1,0,0,-3,-2
C,-1,13,-4,-3,-2,-3,-3,-2,-3,-2,-2,-2,-4,-3,-4,-1,-1,-1,-5,-3
D,-2,-4,8,2,-5,-1,-1,-4,-1,-4,-4,2,-1,0,-2,0,-1,-4,-5,-3
E,-1,-3,2,6,-3,-3,0,-4,1,-3,-2,0,-1,2,0,-1,-1,-3,-3,-2
F,-3,-2,-5,-3,8,-4,-1,0,-4,1,0,-4,-4,-4,-3,-3,-2,-1,1,4
G,0,-3,-1,-3,-4,8,-2,-4,-2,-4,-3,0,-2,-2,-3,0,-2,-4,-3,-3
H,-2,-3,-1,0,-1,-2,10,-4,0,-3,-1,1,-2,1,0,-1,-2,-4,-3,2
I,-1,-2,-4,-4,0,-4,-4,5,-3,2,2,-3,-3,-3,-4,-3,-1,4,-3,-1
K,-1,-3,-1,1,-4,-2,0,-3,6,-3,-2,0,-1,2,3,0,-1,-3,-3,-2
L,-2,-2,-4,-3,1,-4,-3,2,-3,5,3,-4,-4,-2,-3,-3,-1,1,-2,-1


Olhando para a matriz de substituição BLOSUM50, responda:
- Quais valores, os positivos ou os negativos, indicam substituições mais favoráveis?

In [16]:
print(blosum50['A']['G'])
print(blosum50['G']['A'])

print(blosum50['W']['K'])

print(blosum50['A']['A'])
print(blosum50['W']['W'])


0
0
-3
5
15


- Observando os valores a acima, o que mais você pode dizer sobre a matriz BLOSUM?

### Implementando o algoritmo Needleman-Wunsch

Publicado em 1970, o algoritmo realizao alinhamento de um par de sequências biológicas de forma global - indo do primeiro ao último aminoácido ou base nucleotídica.

In [17]:
from skbio import Protein

seqA = Protein("HEAGAWGHEE")
seqB = Protein("PAWHEAE")

print(seqA)
print(seqB)

HEAGAWGHEE
PAWHEAE


In [18]:
# Crie matrizes vazias
num_linhas = len(seqB) + 1
num_colunas = len(seqA) + 1

# Uma para o cálculo da maior diagonal (F)
F = np.zeros(shape=(num_linhas, num_colunas), dtype=int)
show_F(seqA, seqB, F)

,,H,E,A,G,A,W,G,H,E,E
,0,0,0,0,0,0,0,0,0,0,0
P,0,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0,0,0
W,0,0,0,0,0,0,0,0,0,0,0
H,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0
A,0,0,0,0,0,0,0,0,0,0,0
E,0,0,0,0,0,0,0,0,0,0,0


Percebe algo de diferente? Aqui a matriz F tem uma linha e uma coluna a mais, no início das sequências. No algoritmo é importante possuir uma linha e coluna independentes do resíduo inicial.

In [19]:
from intro_bioinfo_unir.iab import show_T

# Outra para a transcrição das sequências alinhadas (T)
T = np.full(shape=(num_linhas, num_colunas), fill_value=" ", dtype=str)
show_T(seqA, seqB, T)

,,H,E,A,G,A,W,G,H,E,E
,,,,,,,,,,,
P,,,,,,,,,,,
A,,,,,,,,,,,
W,,,,,,,,,,,
H,,,,,,,,,,,
E,,,,,,,,,,,
A,,,,,,,,,,,
E,,,,,,,,,,,


Os valores na primeira coluna e linha serão inicializados da seguinte forma:

$F(0,0) = 0$ 

$F(i,0) = F(i-1,0) - d$

$F(0,j) = F(0,j-1) - d$

Sendo $d$ um valor para penalização por inserção de gaps, esse valor será subtraído do escore sempre que um gap for inserido no alinhamento. Aqui vamos usar `d=8` e assim temos:

In [20]:
d = 8
F[0][0] = 0
for i in range(1, num_linhas):
    F[i][0] = F[i-1][0] - d

for j in range(1, num_colunas):
    F[0][j] = F[0][j-1] - d

show_F(seqA, seqB, F)

,,H,E,A,G,A,W,G,H,E,E
,0,-8,-16,-24,-32,-40,-48,-56,-64,-72,-80
P,-8,0,0,0,0,0,0,0,0,0,0
A,-16,0,0,0,0,0,0,0,0,0,0
W,-24,0,0,0,0,0,0,0,0,0,0
H,-32,0,0,0,0,0,0,0,0,0,0
E,-40,0,0,0,0,0,0,0,0,0,0
A,-48,0,0,0,0,0,0,0,0,0,0
E,-56,0,0,0,0,0,0,0,0,0,0


- De quem dependem os escores da primeira coluna? E os da primeira linha?

![]()

Isso pode ser representado na matriz T da seguinte maneira:

In [21]:
T[0][0] = "•"
for i in range(1, num_linhas):
    T[i][0] = "↑"

for j in range(1, num_colunas):
    T[0][j] = "←"

show_T(seqA, seqB, T)

,,H,E,A,G,A,W,G,H,E,E
,•,←,←,←,←,←,←,←,←,←,←
P,↑,,,,,,,,,,
A,↑,,,,,,,,,,
W,↑,,,,,,,,,,
H,↑,,,,,,,,,,
E,↑,,,,,,,,,,
A,↑,,,,,,,,,,
E,↑,,,,,,,,,,


Para computar os valores das demais células, o algoritmo deve encontrar o maior valor possíveis dentre três possibilidades que se baseiam nas células ao redor.

\begin{equation}
       F(i,j) = \max
       \begin{pmatrix}
           F(i-1,j-1) + s(c_i,c_j) \\
           F(i-1,j) - d \\
           F(i,j-1) - d
       \end{pmatrix}
\end{equation} 

Em que $s$ é a matriz de substituição e $c_i$ e $c_j$ são os resíduos na `seqA` e `seqB`.

É esse o trabalho que será realizado pela função a seguir:

In [22]:
from skbio.alignment._pairwise import _compute_score_and_traceback_matrices
from skbio.alignment import TabularMSA

seqA = TabularMSA([seqA])
seqB = TabularMSA([seqB])

matriz_nw, matriz_rastreio = _compute_score_and_traceback_matrices(seqA, seqB, 8, 8, blosum50)

show_F(seqA[0], seqB[0], matriz_nw)

,,H,E,A,G,A,W,G,H,E,E
,0,-8,-16,-24,-32,-40,-48,-56,-64,-72,-80
P,-8,-2,-9,-17,-25,-33,-41,-49,-57,-65,-73
A,-16,-10,-3,-4,-12,-20,-28,-36,-44,-52,-60
W,-24,-18,-11,-6,-7,-15,-5,-13,-21,-29,-37
H,-32,-14,-18,-13,-8,-9,-13,-7,-3,-11,-19
E,-40,-22,-8,-16,-16,-9,-12,-15,-7,3,-5
A,-48,-30,-16,-3,-11,-11,-12,-12,-15,-5,2
E,-56,-38,-24,-11,-6,-12,-14,-15,-12,-9,1


In [23]:
show_T(seqA[0], seqB[0], matriz_rastreio)

,,H,E,A,G,A,W,G,H,E,E
,•,←,←,←,←,←,←,←,←,←,←
P,↑,↖,↖,←,←,←,←,←,←,←,←
A,↑,↖,↖,↖,←,←,←,←,←,←,←
W,↑,↑,↑,↖,↖,←,↖,←,←,←,←
H,↑,↖,↖,↖,↖,↖,↑,↖,↖,←,←
E,↑,↑,↖,←,↖,↖,↖,↑,↖,↖,←
A,↑,↑,↑,↖,←,↖,↖,↖,↑,↑,↖
E,↑,↑,↖,↑,↖,↖,↖,↖,↖,↖,↖


In [24]:
from skbio.alignment._pairwise import _traceback

aln1, aln2, score, _, _ = _traceback(matriz_rastreio,matriz_nw,seqA,seqB, matriz_nw.shape[0]-1, matriz_nw.shape[1]-1)

print(aln1[0])
print(aln2[0])
print(score)

HEAGAWGHE-E
-PA--W-HEAE
1.0


Uma função para realizar todo esse trabalho já está impementada no skbio, e chama-se global_pairwise_align:

In [25]:
from skbio.alignment import global_pairwise_align

aln, score, _ = global_pairwise_align(Protein("HEAGAWGHEE"), Protein("PAWHEAE"), 8, 8, blosum50, penalize_terminal_gaps=True)

print(aln)
print(score)

TabularMSA[Protein]
----------------------
Stats:
    sequence count: 2
    position count: 11
----------------------
HEAGAWGHE-E
-PA--W-HEAE
1.0


/home/tayna/.cache/pypoetry/virtualenvs/intro-bioinfo-unir-DNHDuZ-q-py3.8/lib/python3.8/site-packages/skbio/alignment/_pairwise.py:595: EfficiencyWarning: You're using skbio's python implementation of Needleman-Wunsch alignment. This is known to be very slow (e.g., thousands of times slower than a native C implementation). We'll be adding a faster version soon (see https://github.com/biocore/scikit-bio/issues/254 to track progress on this).
  warn("You're using skbio's python implementation of Needleman-Wunsch "


Percebe esse Warning? Há ainda uma implementação mais rápida do algoritmo.

No caso do alinhamento local, podemos utilizar sequências com sobreposições parciais em suas sequências para identificar a melhor forma de alinhar ao menos uma parte dessa sequência. Uma aplicação de algoritmos de alinhamento local é o [servidor web BLAST](http://blast.ncbi.nlm.nih.gov/Blast.cgi), onde enviamos uma sequência que é comparada com um banco de referência padrão ou de nossa escolha. e recebemos uma ou mais sequências similares, com diversas métricas para avaliação dos alinhamentos.

Você pode seguir esse tutorial na referência, a partir do tópico *Smith-Waterman local sequence alignment* do [capítulo de alinhamento](https://readiab.org/pairwise-alignment.html). Por agora é isso!

![Bye!](https://media3.giphy.com/media/mP8GermRyOFWV8PQeq/giphy.gif?cid=ecf05e476h1el4ct81t72tnychc2t2dl2ukhdzbat0qkpdk2&rid=giphy.gif&ct=g)

## Referência

(2021) **J Gregory Caporaso** [An Introduction to Applied Bioinformatics](https://readiab.org/introduction.html)